# 7. Interact with Excel from Python and vice versa

There are multiple packages in the Python ecosystem that can talk to Excel. 

Furthermore, many packages provide a way to import and export data to/from Excel. Pandas for instance, can export a dataframes as a Excel file and import a range of cells into a dataframe. This is however quite different from actually having a "live" connection between Python and Excel that can listen to individual cell values, run custom function etc. 

**This session focuses on the _xlwings_ ("Make Excel fly") 3rd-party library**, which creates a "live link" between Excel and Python. *xlwings* is free, open source and frequently updated as opposed to some other similar libraries.

> *This type of interaction between Excel and Python is best shown with an instance of both Excel and Python running at the same time. Therefore, the explanations given below should be seen as if there is an Excel spreadsheet running along side of the Python code shown.*

> ***xlwings version:*** *The latest xlwings version at the time of writing this document is v0.18.0 and that is the version used in the Session code and the exercises. Both the session code and the exercises have been tested on v0.17.1 in an Anaconda environment as well. Version other than these might not work directly with the material.*

There's fundamentally two ways you can use this library:
* Manipulate Excel from Python
* Use Python's features from Excel

## A note on standard Python vs Anaconda

The standard version of Python and the Anaconda distribution use two different package management systems.

### PyPI (*pip*)
The standard Python installation uses [PyPI](https://pypi.org/), the Python Package Index. Everybody can upload their packages there for others to use. Once a package is on PyPI it can be easily downloaded via the *pip* command line interface.

### Anaconda (*conda*)
Anaconda is a different Python distribution, which comes with Python installed and it has its own Package Index system. This means that the author(s) of a package need to put it on the Conda Package Index in order for others to be able to install it with `conda install package_name`. 

Some packages are on PyPI, but not on the Conda Package Index, and some versions differ between the two systems.

### *xlwings*

The *xlwings* version on the Conda package system currently one version behind at v0.17.1 compared to the newest v0.18.0 on PyPI. Most likely, at any given time, the PyPI package version will be the same or newer than the Anaconda version.

You can check the documentation for *xlwings* for any released version [here](https://docs.xlwings.org/en/stable/index.html).

#### Some *pip* commands

You can check your current version of *xlwings* by

```code
pip show xlwings
```

If you already have *xlwings* installed, you can upgrade it with *pip* by

```code
pip install -U xlwings
```
Or if you want to install a specific version

```code
pip install xlwings==0.18.0
```

#### Some *conda* commands

You can check your current version of *xlwings* by

```code
conda list xlwings
```

If you already have *xlwings* installed, you can update it with *conda* by 
```
conda update xlwings
```

Or if you want to install a specific version
```code
conda install==0.17.1
```

> **Note:** It is possible to install from *pip* in an Anaconda environment, but the preferred way is to use *conda* to avoid environment inconsistencies.

## Installation

Install the *xlwings* add-in by

```code
pip install xlwings
```

Or if you use the Anaconda distribution

```code
conda install xlwings
```

> **Note:** The *xlwings* version on the `conda` package system currently way out of date (v0.15.4) compared to the newest v0.18.0 in `pip`. That is 17 releases behind! 
This means that the API works in a very different way and the code and exercises below will not be compatible. If you want to use `conda`, you can check the *xlwings* version by `conda list` and find the [Documentation](https://docs.xlwings.org/en/stable/index.html) for that specific version.

> Remember to use the Anaconda Prompt to install!


## Part 1 - Manipulate Excel from Python

In [2]:
import xlwings as xw

### Create a workbook

When we work with the *xlwings* API we deal with objects. This is common for many Python packages.

Internally in *xlwings*, a ***workbook*** is implemented as a class. Let's try to create an instance of that class we can interact with.

In [3]:
# This will create a new workbook
wb = xw.Book()

# Print the class instance and its type
print(wb)
print(type(wb))

<Book [Book1]>
<class 'xlwings.main.Book'>


Notice how executing this code actually made Excel startup with a new empty workbook.

### Referring to sheets
Similar to workbooks are sheets also implemented as a class in xlwings. Let's see what sheets exist inside our newly created workbook.

In [4]:
# Extract the sheets from the workbook
shts = wb.sheets

# Let's inspect what `shts` actually is
print(shts)
print(type(shts))

Sheets([<Sheet [Book1]Sheet1>])
<class 'xlwings.main.Sheets'>


Note that the above refers to all the sheets in the workbook. In this case we just see the default startup sheets.

If we want to interact with a specific sheet, say `Sheet1`, we can extract it directly:

In [5]:
# Extract specific sheet
sht = shts['Sheet1']

# Print to inspect
print(sht)
print(type(sht))

<Sheet [Book1]Sheet1>
<class 'xlwings.main.Sheet'>


### Referring to single cells
Now lets try to interact on a cell level inside the specific sheet selected above.
Values defined/calculated in Python can be inserted into a cell by:

In [6]:
# Assign cell values
sht.range('A1').value = 1    # Insert 1 into cell A1
sht.range('A2').value = 2.5
sht.range('A3').value = 5

To insert a formula you just insert the corresponding string into a cell:

In [7]:
# Insert 'sum' formula
sht.range('A4').value = '=sum(A1:A3)'

Extracting values from Excel to Python is done in a very similar way. 
Let's extract the result of the formula we just inserted above:

In [8]:
# Extract value calculated by formula
A4_value = sht.range('A4').value
print(f'A4 = {A4_value}')

A4 = 8.5


There's serveral ways to refer to a cell. Below we e.g. use the offset option.

In [9]:
# Alternative way to refer to A4 by use of offset
A4_value = sht.range('A1').offset(row_offset=3, column_offset=0).value
print(f'A4 = {A4_value}')

A4 = 8.5


This might seem quite insignificant at a glance, but this concept can be used to bring everything that Python offers into Excel.

### Referring to multiple cells
Python lists can also be inserted into a "cell" by:

In [10]:
# Insert 0 through 9 into cells B1:K2 (i.e. horizontally)
sht['B1'].value = list(range(10))

Notice that the list is inserted horizontally by default. To insert it vertically instead, do this:

In [11]:
# Insert 0 through 9 into cells B2:B11 (i.e. vertically)
sht['B2'].options(transpose=True).value = list(range(10))

Just like you can insert a Python list into an array of cells, you can also extract the values of an array of cells into a Python list:

In [12]:
# Extract range from cells A4:A7
a1_to_a4 = sht['A1:A4'].value
print(a1_to_a4)

[1.0, 2.5, 5.0, 8.5]


### Saving and closing workbook

In [13]:
# Save the workbook
wb.save('Session 7 - Test workbook.xlsx')

In [14]:
# Close the workbook
wb.close()

### Opening existing workbook

In [15]:
# Create a new workbook instance
wb = xw.Book('Session 7 - Test workbook.xlsx')

In [16]:
# You can also just close the whole application (this will exit all involved workbooks without saving!)
wb.app.quit()

### Take control of the whole Excel instance
We're not limited to referring to workbooks and sheets. Here we'll take control of an whole Excel instance. This can for example be useful if you need to loop through a lot of Excel documents and you don't want to wait for Excel to restart every time.

In [17]:
# Start a new instance of Excel
app = xw.App(visible=False, add_book=False)

Application visibility can be set to `False` if the Excel process should run as a *hidden* background process and `add_book` is only specifying if an empty workbook should be initiated or not.

Let's now open the previous workbook using our invisible Excel instance:

In [18]:
# Open an existing workbook with our Excel instance
wb = app.books.open('Session 7 - Test workbook.xlsx')
sht = wb.sheets['sheet1']

We can directly interact with the Excel instance and change its behaviours:

In [19]:
# Turn off screen updating and automatic calculation
app.calculation = 'manual'
app.screen_updating = False

In [20]:
# Change cell 'A1'
sht.range('A1').value = 100

# Extract formula value before having recalculated the SUM formula
A4_value = sht.range('A4').value
print(f'A4 = {A4_value}')

A4 = 8.5


In [21]:
# Turn screen_updating back on and set calculation to automatic
wb.app.screen_updating = True
wb.app.calculation = 'automatic'

Notice how we can also refer to the application of a workbook via `wb.app.`.

In [22]:
# Extract formula value again, now with auto. calculate activated
A4_value = sht.range('A4').value
print(f'A4 = {A4_value}')

A4 = 107.5


It could also be that you would like to turn off any filters before your Excel operations:

In [23]:
# Deactivate filter
if sht.api.AutoFilterMode is True:
    sht.api.AutoFilterMode = False

You can even insert a whole Pandas dataframes by:
```python
# insert dataframe
sht.range('A1').value = df
```

In [24]:
# Close the Excel application (without saving)
app.quit()

Note that even though the application has been closed, the background process are still running.
To fully stop the process you have to delete all references to it by `del app` or by calling:

In [25]:
app.kill()

## Part 2 - Use Python's features from Excel

### Alternative to VBA

The primary selling point for *xlwings* is to provide an alternative to Excel's built-in programming language VBA. The best thing about this is that it enables the user to take advantage of Python's large ecosystem of both built-in language features and third party packages like *Pandas*, *Numpy*, *Scipy*, *Matplotlib* etc.

### Initial setup

#### Create an Excel/Python file pair

This is easiest done by opening the Command Prompt in the directory you want to have the files and running 

```code
xlwings quickstart file_name
```
This should create two files called `file_name.xlsm` and `file_name.py`, which provides an easier setup for the work. Try to avoid spaces in the name.


> ***Hint:*** *You can open the Command Promt (cmd) in a specific folder in multiple ways:*
1. *Open cmd and change directory by `cd path_to_directory`. Get the directory path by copying from file explorer.*
2. *Open cmd directly from file explorer by typing `cmd` in the address line.*


#### Install the *xlwings* add-in for Excel
    
1. Go to the [xlwings releases on GitHub](https://github.com/xlwings/xlwings/releases) and download the file called `xlwings.xlam` from the release matching your installed version of *xlwings*. You can check your version of *xlwings* by `pip show xlwings`. 
  
  
2. Once downloaded, go to Excel => Developer tab => Excel Add-ins => browse for the `xlwings.xlam` file you just downloaded. Pressing "OK" should load the *xlwings* add-in as a ribbon tab in Excel.
To see the Developer tab: File => Options => Customize Ribbon => tick off Developer => OK
        
*See [here](https://docs.xlwings.org/en/stable/addin.html#installation) for the installation instructions from the official documentation.*

#### The two generated files

The generated Excel file is a macro-enabled file (`.xlsm`) with some settings already in place that should have otherwise been done manually.

The generated Python file has some boilerplate code to get started. The file should look like this:

```python
import xlwings as xw

@xw.sub  # only required if you want to import it or run it via UDF Server
def main():
    wb = xw.Book.caller()
    wb.sheets[0].range("A1").value = "Hello xlwings!"


@xw.func
def hello(name):
    return "hello {0}".format(name)


if __name__ == "__main__":
    xw.books.active.set_mock_caller()
    main()
    
```

The first function `main()` is a ***macro*** function, while the second one `hello()` is the User Defined Functions. more about later.

#### Workbook reference when calling Python from Excel

When you call Python code from Excel, the "calling" workbook can be referred to as

```python
wb = xw.Book.caller()
```

This is good to use in that case, since you don't have to hard code file names, which is always something that should be avoided.  

### Macros

The generated Python file from the quickstart had a macro function looking like this:

```python
@xw.sub  # only required if you want to import it or run it via UDF Server
def main():
    wb = xw.Book.caller()
    wb.sheets[0].range("A1").value = "Hello xlwings!"
```

The `@xw.sub` decorator turns the Python function into a VBA macro, which is called a `Sub`.

More about making the macros visible in Excel in the exercises. 

### User defined functions (UDFs)

A User Defined Function (UDF) is something that can be called directly in each cell. `SUM()` is an example of a built-in Excel function.

The generated Python file has an example of that as well:

```python
@xw.func
def greet_name(name):
    """Return a greeting to the inputted name."""
    return f'Hi, {name}'
```

This function type uses a Python decorator as well, this time turning the Python function into an VBA Function, also refered to as an Excel UDF.

There is currently not an exercise in this session which uses UDF's. The API also seem less stable than for macros and the other general interaction shown above.
Refer to the web, especially the *xlwings* [docs](https://docs.xlwings.org/en/stable/), for more information.

## Creating/reading Excel files vs. having a "live link"

As mentioned, there are many Python packages that deal with Excel files. The *xlwings* package as demonstrated above creates a "live session" between an instance of Excel and Python. This is great in many scenarios.

***If the goal however, is only to create/read an Excel file with results***, a package like *openpyxl* or *XlsxWriter* (used by e.g. Pandas) is better suited. Creating Excel files is quite common since Python is good at automating computations and Excel is good for creating the layout of the final report, with nice bordered tables with colors, different fonts etc.

From *openpyxl* all such things can be customized to create a report template. Thus, a live link like *xlwings* offers is not necessary in that case, and actually only complicates things.

Behind the scenes, an Excel file is a bunch of [XML](https://da.wikipedia.org/wiki/XML) files stored in a zipped format. What *openpyxl* does is to write the Excel file in this format to create the resulting workbook. Or read from it if that is the use case. As a result, an Excel installation is not even needed to use *openpyxl*.


## Limitations

While the link between Python and Excel provided by *xlwings* is very convenient, it has some limitations:

- Just like running macros in VBA, Ctrl-Z is not supported for undoing things. This can be quite frustrating.


- It is hard to share the work with colleagues that do not have knowledge about Python or Python installed.


- It can be hard to modularize ones code when working with an Excel/Python interaction like *xlwings*. E.g. if you in a project write multiple useful macros and UDFs, translation to new workbooks/projects is not totally streamlined. How does one create a library of macros and UDFs to seamlessly import into new workbooks? There might be good solutions, in the future or already made, for this though. This sort of thing is much easier done in Python alone.

# Exercises

## Exercise 1 (Setup)

Perform the following tasks

1. Create an Excel/Python file pair by running `xlwings quickstart file_name` from the Prompt. See the section for Initial Setup above for more help. 


2. Create two new sheets called *Sheet2* and *Sheet3* in the workbook.


3. Download the *xlwings* add-in and load it into Excel. See the section for Initial Setup above for more help.


4. The created Python file already has some boiler plate code for importing *xlwings*, a `main()` function and a User Defined Function called `hello()`. 

   * Go ahead and delete the UDF `hello()` with its decorator. We are not going to use it here.

   * From the *xlwings* tab in the Excel ribbon, press the "Run main" button. This should run the `main()` function defined in the Python file.
   
It's succesfull if you now see the text `'Hello World'` in Cell A1 in the first sheet of the workbook.


## Exercise 2

We don't have to use the `main()` function to export values from Excel to Python. You can also do this by running the Python code from the editor.

In the code block `if __name__ == '__main__':`, do the following:

1. Establish a connection to the workbook and save it as a variable, e.g. `wb`.
   
2. Create a variable for referring to *Sheet2*, e.g. `sht2`. 

3. Use NumPy's [arange](https://docs.scipy.org/doc/numpy/reference/generated/numpy.arange.html) function to generate values from -50 to 50 with a step of 5 and insert the resulting 1D-array in cell *A1* in *Sheet2*.

***Hint:*** *Remember to use `.option(transpose=True)` if you want the values to be inserted vertically, your choice.*

***Recall:*** *The `if __name__ == '__main__'` block only gets executed when the Python file is run as a script, ***not*** if is is imported into another Python file. Pressing the "Run main" button executes the file directly (as a script).*


## Exercise 3

In this exercise, we will load a dataset and create a macro to interact with it.

1. **Download a csv file with weather data from Sydney**. You can find the file [here](https://github.com/Python-Crash-Course/Python201/blob/master/Session%207%20-%20Interaction%20with%20Excel/Sydney_weather.csv). Download it by choosing "Raw" => right-click => save as csv. 

    We are going to create a small interface for plotting the dataset. The user should be able to choose

  - Start date
  - End date
  - Parameter to plot
  
  In *sheet3* in the workbook, create input cells for the three things above.


2. Create a new function in the Python file. It should be placed *above* the `if __name__ == '__main__':` and have the `@xw.sub` decorator (signifying a macro). Just leave the function empty for now, we write the code in Step 4.

  After you have named your function in Python, go to Excel => Developer => Visual Basic => Module 1 and add the following VBA code to detect your Python function:
  
```vb
Sub {CHOOSE_MACRO_NAME}()
    mymodule = Left(ThisWorkbook.Name, (InStrRev(ThisWorkbook.Name, ".", -1, vbTextCompare) - 1))
    RunPython ("import " & mymodule & ";" & mymodule & ".{INSERT_PYTHON_FUNCTION_NAME}()")
End Sub
```

Alternatively you can also use the way simpler version where the module name must be entered manually: 
```vb
Sub my_sub()
    RunPython ("import my_module; my_module.my_func()")
End Sub
```

3. In Excel go to Insert => Shape and choose a shape for a button. Right-click the button and assign your macro to it.


4. Write the contents of the function in Step 2. The function has to:
  
   * Read the weather data file into a Pandas dataframe
   * Read the three input values from Excel
   * Filter the dataframe for all input parameters
   * Create a figure object and fill it with a plot of the filtered data
   * Put the resulting figure in *Sheet3* in the workbook
  
If everything is setup correctly, you should now be able to change the dates

---

***Hint:*** You can use this "helper" function for some of the grunt work. Note that only the macro function that in the end interacts with Excel needs to have the `@xw.sub` decorator. Note that you need to `import os` from the standard library to run the function.

```python
def create_daterange_dataframe(filename, start_date, end_date):

    # Get the 'correct' current working dir (where the .py and .xlsm files are)
    cwd = os.path.dirname(os.path.abspath(__file__))

    # Combine into a full path to the dataset file
    full_filename = f'{cwd}\\{filename}'

    df_raw = pd.read_csv(filename)

    # Change the "raw" dates into datetime objects so they can be filtered
    df_raw['Date'] = pd.to_datetime(df_raw['Date'])
    
    # Set the date as the index of the dataframe
    df = df_raw.set_index('Date')

    # Filter dataframe for input date range and return it
    return df[start_date:end_date]
```


***Explanation of the "path" above:*** *When running the Python macro from Excel, the ***current working directory*** might not be set correctly. This might be due to Excel dictating it instead of Python as normally. If this is the case, reading the data file will throw an import error if it is saved in the same directory as you `.xlsm` and `.py` files. The safest bet is to provide the full path of the csv-file. It could be done by this code:*


## Some improvements

Some ideas for improving our little weather plotting app cold be:

- Give the user a more helpful error message if the chosen `parameter` is not present as a column in the dataset. Currently, the dataset cannot be seen from inside Excel, so the user kind of has to "guess" the right string to put in. The default Pandas error is something like `"Keyerror: ..."`.


- Even better than an error message: Read all column name from the dataset and create a drop down in Excel so only valid entries can be chosen.


- Make the plot prettier and more readable.


- Automatically update the plot when either parameter is changed to avoid having to press the button. If not this, at removing/emptying the plot if either parameter change would be good. This will prevent having the temporary "wrong plot" until the button is pressed.


- Every time the "update plot" button is pressed, the code reads in the data as a dataframe and sorts it based on the date range and plotting parameter. This could be improved in many ways, everything from caching previously computed results to using a small database. This will speed up the code a lot and will probably be necessary if the plot should auto-update or if the dataset was larger.

### Other Pandas ideas for interaction with Excel

Some other ideas to try for this dataset:

- Generate descriptive statistics for entire datafrmae: [pandas.DataFrame.describe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html)

- Compute statistics over custom time intervals: [pandas.DataFrame.resample](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html)



# End of exercises
The cell below is for setting the style of this document. It's not part of the exercises.

In [25]:
# Apply css theme to notebook
from IPython.display import HTML
HTML('<style>{}</style>'.format(open('../css/cowi.css').read()))